### **Import Dependecies**

In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

### **Cut the Video into 30 Frames**

In [2]:
def extract_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * total_frames // num_frames)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break

    cap.release()
    return frames

### **Extracting Key Points on Hand Landmarks**

In [29]:
# change with the video data captured from the phone
frames = extract_frames('video/A.mp4')
keypoints_sequence = []

In [30]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for frame in frames:
        _, results = mediapipe_detection(frame, holistic)
        keypoints = extract_keypoints(results)
        keypoints_sequence.append(keypoints)

keypoints_sequence = np.array(keypoints_sequence)

# Expand dimensions to match model's expected input (NEW input for ConvLSTM)
keypoints_sequence = np.expand_dims(keypoints_sequence, axis=-1)  # Shape: (num_frames, keypoints_dim, 1)
keypoints_sequence = np.expand_dims(keypoints_sequence, axis=-1)  # Shape: (num_frames, keypoints_dim, 1, 1)

### **Model predictions**

In [23]:
model = load_model('../model/33Class_ConvLSTM_acc098_loss01_50seq.h5')

# Kelas pada model
actions = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
                    'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 
                    'U', 'V', 'W', 'X', 'Y', 'Z', 'Halo', 'Perkenalkan', 
                    'Nama', 'Saya', 'Senang', 'Bertemu', 'Kamu'])

# Expand dimensions to match model's expected input
keypoints_sequence = np.expand_dims(keypoints_sequence, axis=0)

# Predict
res = model.predict(keypoints_sequence)[0]
predicted_action = actions[np.argmax(res)]

print(f"Predicted action: {predicted_action}")


1/1 [==============================] - 2s 2s/step
Predicted action: Bertemu


### **TEMP CODE**

#### Simpan 30 frame ke folder (untuk cek)

note: kalo bisa semua 30 frame, tangannya udah di dalam frame

In [15]:
def extract_and_save_frames(video_path, output_folder, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * total_frames // num_frames)
        ret, frame = cap.read()
        if ret:
            frame_filename = os.path.join(output_folder, f'frame_{i + 1}.jpg')
            cv2.imwrite(frame_filename, frame)
        else:
            print(f"Failed to read frame {i}")
            break

    cap.release()
    print(f"Saved {num_frames} frames to {output_folder}")

# Path ke video yang akan diekstrak
video_path = 'video/C-pakeHP.mp4'

# Nama folder output
output_folder = 'frame-video'

# Ekstrak dan simpan frame
extract_and_save_frames(video_path, output_folder)

Saved 30 frames to frame-video-t


In [27]:
def make_directory(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

def extract_frames(video_path, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * total_frames // num_frames)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break

    cap.release()
    return frames

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_landmarks(image, results):
    mp_drawing = mp.solutions.drawing_utils
    mp_holistic = mp.solutions.holistic

    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

def save_landmarked_frames(frames, output_folder, holistic):
    make_directory(output_folder)

    for i, frame in enumerate(frames):
        image, results = mediapipe_detection(frame, holistic)
        draw_landmarks(image, results)
        frame_filename = os.path.join(output_folder, f'frame_landmark_{i + 1}.jpg')
        cv2.imwrite(frame_filename, image)
        print(f"Saved frame {i + 1} with landmarks to {frame_filename}")

# Path ke video yang akan diekstrak
video_path = 'video/A.mp4'

# Nama folder output
output_folder = 'landmark'

# Ekstrak frame dari video
frames = extract_frames(video_path, num_frames=30)

# Deteksi dengan Mediapipe dan simpan gambar yang sudah ada landmark
mp_holistic = mp.solutions.holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

save_landmarked_frames(frames, output_folder, mp_holistic)
print(f"Saved all frames to {output_folder}")

Saved frame 1 with landmarks to landmark-Bertemu\frame_landmark_1.jpg
Saved frame 2 with landmarks to landmark-Bertemu\frame_landmark_2.jpg
Saved frame 3 with landmarks to landmark-Bertemu\frame_landmark_3.jpg
Saved frame 4 with landmarks to landmark-Bertemu\frame_landmark_4.jpg
Saved frame 5 with landmarks to landmark-Bertemu\frame_landmark_5.jpg
Saved frame 6 with landmarks to landmark-Bertemu\frame_landmark_6.jpg
Saved frame 7 with landmarks to landmark-Bertemu\frame_landmark_7.jpg
Saved frame 8 with landmarks to landmark-Bertemu\frame_landmark_8.jpg
Saved frame 9 with landmarks to landmark-Bertemu\frame_landmark_9.jpg
Saved frame 10 with landmarks to landmark-Bertemu\frame_landmark_10.jpg
Saved frame 11 with landmarks to landmark-Bertemu\frame_landmark_11.jpg
Saved frame 12 with landmarks to landmark-Bertemu\frame_landmark_12.jpg
Saved frame 13 with landmarks to landmark-Bertemu\frame_landmark_13.jpg
Saved frame 14 with landmarks to landmark-Bertemu\frame_landmark_14.jpg
Saved fram